# Representation Generator for the h-space similarity explorer

In [ ]:
from diffusers import AutoPipelineForText2Image, AutoencoderKL
import torch
import numpy as np
from pathlib import Path
import json
from PIL import Image
from tqdm.notebook import tqdm
from contextlib import ExitStack
from functools import partial
from typing import Any

## Config

In [ ]:
models: list[dict[str,Any]] = [
    dict(
        short='SDXL-Turbo',
        name='stabilityai/sdxl-turbo',
        steps=4,
        guidance_scale=0.0,
        extract_positions = ['conv_in', 'down_blocks[0]', 'down_blocks[1]', 'down_blocks[2]', 'mid_block', 'up_blocks[0]', 'up_blocks[1]', 'up_blocks[2]', 'conv_out'],
    ),
    dict(
        short='SD-Turbo',
        name='stabilityai/sd-turbo',
        steps=2,
        guidance_scale=0.0,
        extract_positions = ['conv_in', 'down_blocks[0]', 'down_blocks[1]', 'down_blocks[2]', 'mid_block', 'up_blocks[0]', 'up_blocks[1]', 'up_blocks[2]', 'conv_out'],
    ),
    dict(
        short='SD-1.5',
        name='runwayml/stable-diffusion-v1-5',
        steps=50,
        guidance_scale=7.5,
        extract_positions = ['down_blocks[1]', 'down_blocks[2]', 'mid_block', 'up_blocks[0]'],
    ),
]

prompts = {
    "Cat": "A photo of a cat.",
    "Dog": "A photograph of a husky, dog, looking friendly and cute.",
    "Polarbear": "A photo of a polar bear.",
    "ConstructionWorker": "A photo of a hard working construction worker.",
    "Woman": "A photo of a beautiful, slightly smiling woman in the city.",
    "OldMan": "A portrait of an old man with a long beard and a hat.",
    "FuturisticCityscape": "A futuristic cityscape at sunset, with flying cars and towering skyscrapers, in the style of cyberpunk.",
    "MountainLandscape": "A serene mountain landscape with a crystal-clear lake in the foreground, reflecting the snow-capped peaks under a bright blue sky.",
    "SpaceAstronaut": "A high-res photo of an astronaut floating in the vastness of space, with a colorful nebula and distant galaxies in the background.",
    "MajesticLion": "A close-up portrait of a majestic lion, with detailed fur and piercing eyes, set against the backdrop of the African savannah at dusk.",
    "MagicalForest": "A magical forest filled with glowing plants, mythical creatures, and a pathway leading to an enchanted castle.",
    "JapaneseGarden": "A traditional Japanese garden in spring, complete with cherry blossoms, a koi pond, and a wooden bridge.",
}

seed = 0

## Generate Representations

In [ ]:
def get_reprs(pipe, prompt, steps, guidance_scale, extract_positions):
    '''Get representations and intermediate images from a model.'''
    representations = {pos: [] for pos in extract_positions}
    images = []
    def latents_callback(i, t, latents):
        latents = 1 / pipe.vae.config.scaling_factor * latents.to(next(iter(pipe.vae.post_quant_conv.parameters())).dtype)
        image = pipe.vae.decode(latents).sample[0]
        image = (image / 2 + 0.5).clamp(0, 1)
        image = image.cpu().permute(1, 2, 0).numpy()
        images.extend(pipe.numpy_to_pil(image))
    
    with ExitStack() as stack, torch.no_grad():
        for extract_position in extract_positions:
            def get_repr(module, input, output, extract_position):
                representations[extract_position].append(output[0].cpu().numpy())
            # eval is unsafe. Do not use in production.
            stack.enter_context(eval(f'unet.{extract_position}', {'__builtins__': {}, 'unet': pipe.unet}).register_forward_hook(partial(get_repr, extract_position=extract_position)))
        result = pipe(prompt, num_inference_steps = steps, guidance_scale=guidance_scale, callback=latents_callback, callback_steps=1, generator=torch.Generator("cuda").manual_seed(seed))
    return representations, images

# setup output directory
base_path = Path('representations')
base_path.mkdir(exist_ok=True)
with open(base_path / '.gitignore', 'w') as f:
    f.write('*')

# run the models
for model_dict in models:
    model_name = model_dict['name']
    model_name_short = model_dict['short']

    # load model
    pipe = AutoPipelineForText2Image.from_pretrained(model_name, torch_dtype=torch.float16).to('cuda')
    pipe.set_progress_bar_config(disable=True)  # disable progress bar
    if hasattr(pipe, 'upcast_vae') and pipe.vae.dtype == torch.float16 and pipe.vae.config.force_upcast:
        pipe.upcast_vae()  # upcast VAE to float32 if needed (required for SDXL models)

    # note model h-space dimensions
    representations, _ = get_reprs(pipe, '', 1, 0, model_dict['extract_positions'])
    model_dict['representations'] = {}
    for pos, reprs in representations.items():
        model_dict['representations'][pos] = {
            'channels': reprs[0].shape[-3],
            'spatial': reprs[0].shape[-1],
        }

    # go through prompts
    for i, (prompt_name, prompt) in enumerate(tqdm(prompts.items(), desc=f'Running {model_name_short}')):

        # setup save path
        save_path = base_path / model_name_short / prompt_name
        save_path.mkdir(exist_ok=True, parents=True)

        # run the model
        representations, images = get_reprs(pipe, prompt, model_dict['steps'], model_dict['guidance_scale'], model_dict['extract_positions'])

        # save representations
        for pos, reprs in representations.items():
            with open(save_path / f'repr-{pos}.bin', 'wb') as f:
                f.write(np.array(np.stack(reprs), dtype=np.float32).tobytes())

        # save intermediate images
        for j, img in enumerate(images, 1):
            img.save(save_path / f'{j}.jpg')

        # save config
        git_hash = !git rev-parse main
        with open(save_path / 'config.json', 'w') as f:
            f.write(json.dumps({**model_dict, 'prompt_name': prompt_name, 'prompt': prompt, 'git_hash': git_hash[0], 'seed': seed}))

# save global config files
with open(base_path/'prompts.json', 'w') as f:
    f.write(json.dumps(prompts))
with open(base_path/'models.json', 'w') as f:
    f.write(json.dumps(models))